# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import googlemaps

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

### Use csv from Weather Py 

In [ ]:
weather_file = pd.read_csv("../output_data/city.csv")
weather_file.dropna()
weather_file


### Humidity Heatmap


In [ ]:
#Load gmaps gkey
gmaps.configure(api_key=g_key)

In [ ]:

locations = weather_file[["Lat", "Lng"]]
humidity = weather_file["Humidity"].astype(float)

In [ ]:
hfigure = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
hfigure.add_layer(heat_layer)
#Display figure
hfigure

### Cities meeting ideal weather conditions

In [ ]:
temp_file=weather_file.loc[(weather_file['Max Temp']>=60)&(weather_file['Max Temp']<=90)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']<=10)]
ideal_weather



### Hotel Map

In [ ]:
possible_hotels = ideal_weather.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
possible_hotels["Hotel Name"] = ""
possible_hotels

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [ ]:
for index, row in possible_hotels.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    json.dumps(hotel_name, indent=4, sort_keys=True)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        possible_hotels.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        
possible_hotels

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in possible_hotels.iterrows()]
locations = possible_hotels[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

hfigure.add_layer(markers)

# Display Map
hfigure
